In [ ]:
from collections import defaultdict
import os
from irods.session import iRODSSession
from irods.models import Collection, DataObject
from irods.exception import CollectionDoesNotExist

In [ ]:
def get_project_sample_collection_paths():
    project_to_sample_collections = defaultdict(list)
    
    env_file = os.path.expanduser('~/.irods/irods_environment.json')
    with iRODSSession(irods_env_file=env_file) as irods_session:
        # from the top
        top = '/iplant/home/shared/imicrobe/projects'
        project_collections = irods_session.collections.get(top).subcollections
        for project_collection in project_collections:
            samples_collection_path = os.path.join(project_collection.path, 'samples')
            try:
                sample_collections_for_project = irods_session.collections.get(samples_collection_path).subcollections
                print('{} sample collection(s) for project {}'.format(
                    len(sample_collections_for_project),
                    project_collection.name))
                    #'\n\t'.join([s.name for s in sample_collections_for_project])))
            except CollectionDoesNotExist:
                print('collection "{}" does not exist'.format(samples_collection_path))
                sample_collections_for_project = []
            project_to_sample_collections[project_collection.path].extend(
                [s.path for s in sample_collections_for_project])

        projects_with_no_sample_collections = [
            p
            for p, s
            in project_to_sample_collections.items()
            if len(s) == 0]
        print('{} projects with no sample collections:\n\t{}'.format(
            len(projects_with_no_sample_collections),
            '\n\t'.join(projects_with_no_sample_collections)))

    return project_to_sample_collections

In [ ]:
project_to_sample_collections = get_project_sample_collection_paths()

print('sample collection paths:\n\t{}'.format('\n\t'.join([
    p.path + ':' + '\n\t'.join(s)
    for
    p, s
    in project_to_sample_collections.items()])))

print('projects without sample collections:\n\t{}'.format(
    '\n\t'.join([
        p
        for
        p, s
        in project_to_sample_collections.items()
        if len(s) == 0])))